# 📘 Tutorial 4: Tensor Gradients and Vector–Jacobian Products
> This notebook is provided in a clean, non-executed state for readability and reproducibility.
>
> A model answer and executive summary for the practical exercise can be found in the *worked* version.

This tutorial builds on Tutorials 1–3 and deepens the understanding of PyTorch autograd by moving beyond scalar losses to tensor-valued outputs.

In many realistic settings, computations do not naturally terminate in a single scalar. When the output is a tensor, the notion of “the gradient” must be refined. This tutorial introduces **vector–Jacobian products (VJPs)** as the fundamental object computed by `autograd` when calling `backward()` on non-scalar outputs.

---
**Rather than treating `backward()` as a black-box operation, the tutorial reframes it as a precise mathematical request:**
- not *“give me the gradient”*,
- but *“how does a weighted combination of outputs depend on earlier tensors?”*
---
**The emphasis is on developing intuition for**:
- how gradients are defined for tensor-valued functions,
- why full Jacobians are rarely constructed explicitly,
- how explicit upstream gradients select directions in output space,
- and how `.grad` should be interpreted as a **sensitivity measure**, not a loss signal.
---
**Key ideas explored include**:
- the distinction between scalar losses and tensor-valued outputs,
- vector–Jacobian products as the object computed by `backward(v)`,
- the role of upstream gradients in shaping gradient flow,
- gradient accumulation and the need for explicit zeroing,
- and interpreting gradient structure through controlled visual experiments.
---
**This tutorial serves as a conceptual bridge between**:
- computation graph mechanics and backpropagation (Tutorial 3),
- and gradient interpretation and objective design (Workshop 1).

The focus is intentionally not on optimisation algorithms or training loops, but on **understanding what gradients mean** before they are used to update parameters.

---
**Recommended prerequisites**:
- Familiarity with PyTorch tensors and `.backward()`
- Understanding of computation graphs and leaf tensors
- Basic comfort with linear algebra and partial derivatives
---
**Author**: Angze Li

**Last updated**: 2026-02-20

**Version**: v2.0

In [ ]:
import torch

## 1. From scalar losses to tensor-valued outputs

In most machine learning workflows, we work with a scalar loss function and compute gradients with respect to model parameters.
This scalar setting allows PyTorch to apply backpropagation automatically, storing gradients directly in `.grad`.

However, not all computations naturally end in a scalar.

In many practical and analytical settings, the output of interest is an arbitrary **tensor** rather than a single number. In such cases, a full gradient is no longer *well-defined* in the usual sense. Instead, the derivative of a tensor-valued function is a **Jacobian matrix**, which can be large and expensive to construct explicitly.

Rather than forming this Jacobian, PyTorch computes Jacobian products efficiently.

Specifically, PyTorch evaluates **vector–Jacobian products** (VJPs), which answer the question:

> How does a weighted combination of the output change with respect to earlier tensors?

In [ ]:
inp = torch.eye(4, 5, requires_grad=True)
out = (inp+1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"leaf tensor inp\n{inp}")
print(f"out\n{out}")
print(f"First call\n{inp.grad}")

### What does the code mean?

#### First Line

```python
torch.eye()
```
creates a identity-like matrix (ones on the diagonal, zeros elsewhere). 

By passing `requires_grad=True`, we tell PyTorch to track all subsequent operations on this tensor so that gradients can be computed during backpropagation.

#### Second Line

The variable `out` is now a **tensor-valued output**, not a scalar:
```python
out = (inp+1).pow(2).t()
```
This line performs several operations in sequence:
1. `inp + 1` adds 1 element-wise to the input tensor.
2. `.pow(2)` squares each element, introducing a nonlinear operation (common in machine learning models).
3. `.t()` transposes the matrix, changing its shape.

As a result, out is a matrix, not a single number.

#### Third Line: why does `backward()` need an argument here?

In earlier tutorials, we always called backward() on a scalar loss. In that case, PyTorch *implicitly* assumes an upstream gradient of 1.

Here, however, out is not a scalar. Calling:
```python
out.backward()
```
would raise an error, because PyTorch does not know which **combination of gradients** you want.

Instead, we must explicitly provide an upstream gradient with the **same shape** as `out`:
```python
out.backward(torch.ones_like(out), retain_graph=True)
```
This tells PyTorch:
> “Compute the vector–Jacobian product where the upstream gradient is a tensor of ones.”


#### Final line: What is `inp.grad` showing us?

After calling:
```python
out.backward(torch.ones_like(out), retain_graph=True)
```
PyTorch computes gradients **with respect to all leaf tensors** that have `requires_grad=True`.
In this example, the only such tensor is:
```python
inp
```
As a result, PyTorch stores the gradient of the output `out` with respect to `inp` in:
```python
inp.grad
```

#### Wait a second...

This could be counter-intuitive for beginners, as the `backward()` is called on `out`, but the gradient is stored on `inp`. 

However, the key mental shift is:
> `backward()` is not asking “what is the gradient of this tensor?”
>
> It is asking “how did this output depend on earlier tensors?”

In PyTorch, when you call:
```python
out.backward(torch.ones_like(out))
```
you are telling PyTorch:
> “Assume out represents the end of a computation.
> 
>Please compute how changes in earlier tensors would affect it.”

More precisely, PyTorch computes:
$$\frac{\partial}{\partial (\text{leaf tensors})}
\Big(v_{\text{upstream}}^{\top}\cdot\text{out}\Big)$$
where:
- $v_{\text{upstream}}$ is the upstream gradient supplied to `backward()` (in our case, `torch.ones_like(out)`, which corresponds to summing all entries of out with **equal weight**; other choices are also valid).
- The expression $v_{\text{upstream}}^{\top}\cdot\text{out}$ is equivalent to $\sum_i v_{\text{upstream},i}\,\text{out}_i$, giving a scalar as the result. This is called **gradient produced via a vector–Jacobian product (VJP)**.
- Only the leaf tensors (see definition in Tutorial 2) accumulate gradients in their `.grad` attribute during backpropagation (in our case, `inp` is the (only) leaf tensor, and hence PyTorch stores the result of the above derivative in `inp.grad`).

Note that mathematically, the gradient (partial derivatives) of a scalar $L$ with respect to a matrix $X \in \mathbb{R}^{m\times n}$ is defined as:
$$\frac{\partial L}{\partial X}
\;=\;
\begin{bmatrix}
\frac{\partial L}{\partial X_{11}} & \cdots & \frac{\partial L}{\partial X_{1n}} \\
\vdots & \ddots & \vdots \\
\frac{\partial L}{\partial X_{m1}} & \cdots & \frac{\partial L}{\partial X_{mn}}
\end{bmatrix}$$

There are (at least) two reasons why PyTorch does not store the gradient in `out`:
1. **`out` is not a parameter.** We don’t usually update out; we update inputs like weights.
2. **Gradients are defined with respect to inputs.** The whole purpose of backpropagation is to answer: “Which earlier values should I change to reduce the output?”

#### Important clarification

The need to pass an argument to `backward()` here is not because we want to call `backward()` multiple times. It is because `out` is non-scalar.

The reason we pass `retain_graph=True` is separate:
- it allows the same computation graph to be reused in later cells,
- which is useful for demonstration and debugging.

We will discuss gradient accumulation and zeroing gradients in the next cells.

## 2. Gradient accumulation and zeroing gradients

In [ ]:
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nSecond call\n{inp.grad}")

if inp.grad is not None:
    inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph=True)
print(f"\nCall after zeroing gradients\n{inp.grad}")

### What is happening here?

PyTorch accumulates gradients by default.
This means that every call to `backward()` adds new gradient contributions to the existing values stored in `.grad`.

When we call `backward()` for a second time:
```python
out.backward(torch.ones_like(out), retain_graph=True)
```
without clearing gradients:
- PyTorch computes the same gradient again
- the newly computed gradient is **added to the existing contents** of `inp.grad`

As a result, `inp.grad` after the second call contains the **sum of gradients** from both backward passes, not just the most recent one (we can find that the result of second call is exactly twice as the first call above).

This behaviour is intentional and is essential for many training workflows (e.g. gradient accumulation over multiple mini-batches).

However, when we intend to reset the gradients, we can use:
```python
inp.grad.zero_()
```
This clears the accumulated gradient and sets all entries of `inp.grad` to zero. Now, when we call
```python
out.backward(torch.ones_like(out), retain_graph=True)
```
again:
- PyTorch computes the gradient from scratch
- the result stored in `inp.grad` now reflects only this single backward pass.

Now the result is the same as the first call.

### Why does PyTorch accumulate gradients?

PyTorch’s design assumes that:
- users may want to accumulate gradients across multiple forward–backward passes,
- or combine gradients from multiple loss terms.

Because of this, gradient clearing is an **explicit** user responsibility.

This is why, in typical training loops, you will often see:
```python
optimizer.zero_grad()
```
or 
```python
tensor.grad.zero_()
```
before each backward pass.

## 3.  Practical exercise: weighted sensitivity of a tensor output


>In many real problems, a model does not produce a single scalar output.
>Instead, we often care about how sensitive different parts of the output are to changes in the input, possibly with different importance weights.

Consider the following setup:
```python
inp = torch.eye(4, 5, requires_grad=True)
out = (inp + 10).pow(3).t()
```
This produces a tensor-valued output.
Suppose now that we care more about *some entries* of `out` than others.

---
#### Task

1. Construct an upstream gradient tensor v with the same shape as out that:
    - assigns higher weight to the first row of out,
    - and lower (or zero) weight elsewhere.
2.	Call:
```python
out.backward(v)
```
3.	Inspect `inp.grad`.

---

#### Questions to think about
- How does changing `v` change `inp.grad`?
- Which entries of `inp` are most sensitive to the weighted output?
- What does `inp.grad` represent conceptually in this case?

---

#### Hint

>You are not computing “the gradient of out”.
>
>You are computing how a weighted combination of outputs depends on the input.

### Solution

## 🧭 Closing Remarks

This tutorial completes the transition from scalar losses to tensor-valued outputs, clarifying what `autograd` computes when a computation does not naturally end in a single number.

By working explicitly with vector–Jacobian products, you have seen that gradients are not intrinsic properties of tensors, but **responses to a chosen direction in output space**. The upstream gradient supplied to `backward(v)` is not a technical nuisance—it is a precise way of defining *what you care about*.

This perspective is essential before moving on to optimisation:
- it explains why gradients depend on objectives, not just models,
- it clarifies how sensitivity is redistributed across inputs,
- and it demystifies the role of `.grad` as a geometric object rather than a training artifact.

In the workshops, these ideas will be pushed further. Instead of merely inspecting gradients, you will begin to **shape them deliberately** and interpret how their structure anticipates optimisation dynamics—without yet introducing optimisers or training loops.

At that point, optimisation will no longer feel like a new concept, but simply the repeated application of principles you already understand.